# Understanding the Interface class

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.6 MB/s eta 0:00:00
   ━━━━━

In [ ]:
import numpy as np
import gradio as gr

# Fungsi untuk membalikkan audio
def reverse_audio(audio):
    sr, data = audio  # Mendapatkan sample rate dan data audio
    reversed_audio = (sr, np.flipud(data))  # Membalikkan data audio secara vertikal
    return reversed_audio

# Komponen audio input tanpa argumen 'source'
audio_input = gr.Audio(type="numpy", label="Speak here...")

# Menggunakan 'inputs' dalam Gradio Interface untuk menentukan sumber
gr.Interface(
    fn=reverse_audio,  # Fungsi untuk membalikkan audio
    inputs=audio_input,  # Input berupa audio
    outputs="audio"  # Output berupa audio
).launch(share=True)  # Menambahkan opsi berbagi aktif


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://308e6ed97610c87c10.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import numpy as np
import gradio as gr

notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

# Fungsi untuk menghasilkan nada

def generate_tone(note, octave, duration):
    sr = 48000  # Sample rate
    a4_freq, tones_from_a4 = 440, 12 * (octave - 4) + (note - 9)  # Frekuensi dasar A4 dan jarak nada
    frequency = a4_freq * 2 ** (tones_from_a4 / 12)  # Menghitung frekuensi nada
    # Memastikan durasi adalah angka
    duration = float(duration)
    audio = np.linspace(0, duration, int(duration * sr))  # Membuat array waktu
    audio = (20000 * np.sin(audio * (2 * np.pi * frequency))).astype(np.int16)  # Membuat sinyal audio
    return (sr, audio)  # Mengembalikan sample rate dan data audio

# Membuat antarmuka Gradio
gr.Interface(
    generate_tone,  # Fungsi utama
    [
        gr.Dropdown(notes, type="index"),  # Dropdown untuk memilih nada
        gr.Slider(minimum=4, maximum=6, step=1, label="Octave"),  # Slider untuk memilih oktaf
        gr.Textbox(value=1, label="Duration in seconds"),  # Kotak teks untuk memasukkan durasi
    ],
    "audio",  # Output berupa audio
).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b1761568986974cf48.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from transformers import pipeline
import gradio as gr

# Memuat model untuk pengenalan ucapan otomatis
model = pipeline("automatic-speech-recognition")

# Fungsi untuk mentranskripsikan audio dari mikrofon atau file
def transcribe_audio(mic=None, file=None):
    if mic is not None:
        audio = mic  # Menggunakan input dari mikrofon jika tersedia
    elif file is not None:
        audio = file  # Menggunakan input dari file jika tersedia
    else:
        return "You must either provide a mic recording or a file"  # Pesan error jika tidak ada input

    transcription = model(audio)["text"]  # Melakukan transkripsi audio menggunakan model
    return transcription

# Membuat antarmuka Gradio untuk mentranskripsikan audio
gr.Interface(
    fn=transcribe_audio,  # Fungsi utama untuk transkripsi
    inputs=[
        gr.Audio(sources=["microphone"], type="filepath", label="Microphone Input"),  # Input dari mikrofon
        gr.Audio(sources=["upload"], type="filepath", label="File Upload"),  # Input dari file
    ],
    outputs="text",  # Output berupa teks hasil transkripsi
).launch()


No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 22aad52 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

Device set to use cpu


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e96e96595fd8f50ade.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
